In [3]:
import pydap
import netCDF4 as nc
import pandas as pd
import h5py
import xarray as xr
import geopy.distance
import numpy as np
import os

In [7]:
f_oco2 = nc.Dataset('gosat/2016/acos_LtCO2_160101_v201201_B7310A_161107211943s.nc4')
f_oco2

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    filter_function: acos_l2_filter_b7
    bc_function: bias_correct_acos_b7
    warn_level_function: acos_warn_levels_b7
    Bias_Correction_landH: XCO2_Bias_Corrected = XCO2_Raw + 0.300*(dp + 0.0) + 8.60*(sqrt_alb_3 - 0.500) + 0.0160*(co2_grad_del - 25.0) + 14.5*(DWS - 0.0200) +0.15 ppm
    Bias_Correction_landM: XCO2_Bias_Corrected = XCO2_Raw + 0.290*(dp + 1.20) + 4.10*(alb_3 - 0.400) + 0.0150*(co2_grad_del - 20.0) + 7.50*(aod_du - 0.0100) -0.20 ppm
    Bias_Correction_Ocean: XCO2_Bias_Corrected = XCO2_Raw - 42.4*(s32 - 0.610) - 0.0930*(co2_grad_del + 3.00) + 1.80*(aodheight_ic - 0.180) + 0.325*(logDust + 0.0) +0.90 ppm
    Bias_Uncertainty: Bias correction parameter uncertainties are given in the data users guide.
    Note_regarding_bias_correction: There is some uncertainty in not only the bias correction coefficients but in the chosen parameters themselves. This release bias correction may be sl

In [10]:
list(f_oco2.variables.keys())

['sounding_id',
 'levels',
 'bands',
 'date',
 'latitude',
 'longitude',
 'time',
 'solar_zenith_angle',
 'sensor_zenith_angle',
 'xco2_quality_flag',
 'source_files',
 'file_index',
 'warn_level',
 'xco2',
 'xco2_uncertainty',
 'xco2_apriori',
 'pressure_levels',
 'co2_profile_apriori',
 'xco2_averaging_kernel',
 'pressure_weight']

In [43]:
features = ['sounding_id',
 'latitude',
 'longitude',
 'time',
 'solar_zenith_angle',
 'sensor_zenith_angle',
 'xco2_quality_flag',
 'xco2',
 'xco2_uncertainty',
 'xco2_apriori',
 'pressure_levels',
 'co2_profile_apriori',
 'pressure_weight']

In [48]:
path = './data_2016_oco2/'
outpath = './csv/'
for file in os.listdir(path):
    if str(file).split('.')[-1]!='nc4': continue
    # new code
    f_oco2 = nc.Dataset(os.path.join(path,file))
    dates =[]
    for dt in f_oco2['date']:
        s = ""
        for num in dt:
            s += str(num)+"/"
        dates.append(s[:-1])
    df_oco2 = pd.DataFrame(columns=features)
    for col in df_oco2.columns:
        df_oco2[col] = [x for x in f_oco2[col]]

    df_oco2["dates"] = dates    
    df_oco2["dates"] = pd.to_datetime(df_oco2["dates"],format='%Y/%m/%d/%H/%M/%S')
    df_oco2.to_csv(os.path.join(path,f'{file}.csv'))


KeyboardInterrupt: 

In [ ]:
path = './gosat/2016'
outpath = './csv/'
i=0
for file in os.listdir(path):
    if str(file).split('.')[-1]!='nc4': continue
    # new code
    f_gosat = nc.Dataset(os.path.join(path,file))
    dates =[]
    for dt in f_gosat['date']:
        s = ""
        for num in dt:
            s += str(num)+"/"
        dates.append(s[:-1])
    df_gosat = pd.DataFrame(columns=features)
    for col in df_gosat.columns:
        df_gosat[col] = [x for x in f_gosat[col]]

    df_gosat["dates"] = dates    
    df_gosat["dates"] = pd.to_datetime(df_gosat["dates"],format='%Y/%m/%d/%H/%M/%S/%f')
    df_gosat.to_csv(os.path.join(path,f'{file}.csv'))


In [156]:
oco_jan1 = pd.read_csv('./data_2016_oco2/csv_folder/oco2_1ndjan2016.csv').drop(columns=['Unnamed: 0'])
oco_jan1['point'] = oco_jan1.apply(lambda x: shapely.Point(x.latitude,x.longitude),axis=1)
oco_jan1.head()

,latitude,longitude,time,sensor_zenith_angle,xco2,xco2_uncertainty,xco2_apriori,dates,point
0,-3.628539,-157.43341,1.451606e+09,22.461530,400.44990,0.393150,399.77917,2016-01-01 00:00:00.000,POINT (-3.6285386 -157.43341)
1,-3.639151,-157.43082,1.451606e+09,22.516706,400.22430,0.386413,399.77840,2016-01-01 00:00:00.000,POINT (-3.6391513 -157.43082)
2,-3.649849,-157.42830,1.451606e+09,22.573343,400.45795,0.384631,399.77770,2016-01-01 00:00:00.000,POINT (-3.6498492 -157.4283)
3,-3.660598,-157.42587,1.451606e+09,22.631292,399.87880,0.381401,399.78036,2016-01-01 00:00:00.000,POINT (-3.6605978 -157.42587)
4,-3.671286,-157.42354,1.451606e+09,22.689938,400.19666,0.379429,399.79030,2016-01-01 00:00:00.000,POINT (-3.6712859 -157.42354)


In [160]:
import shapely

In [161]:
gosat_jan1 = pd.read_csv('./gosat/2016/csv_folder/gosat_1ndjan2016.csv').drop(columns=['Unnamed: 0'])
gosat_jan1['point'] = gosat_jan1.apply(lambda x: shapely.Point(x.latitude,x.longitude),axis=1)
# gosat_jan1 = gosat_jan1.sort_values(by=['latitude','longitude'])
gosat_jan1.head()

,latitude,longitude,time,sensor_zenith_angle,xco2,xco2_uncertainty,xco2_apriori,dates,point
0,-74.99241,162.841260,1.451608e+09,26.974253,401.64148,3.566810,399.33536,2016-01-01 00:33:01,POINT (-74.99241 162.84126)
1,-74.98562,162.859530,1.451608e+09,25.909986,405.42490,3.769866,399.33542,2016-01-01 00:33:06,POINT (-74.98562 162.85953)
2,-74.97762,162.877980,1.451608e+09,25.049591,410.68390,4.687877,399.33542,2016-01-01 00:33:11,POINT (-74.97762 162.87798)
3,-74.41205,141.735400,1.451608e+09,26.970304,404.74084,3.405084,399.04263,2016-01-01 00:33:43,POINT (-74.41205 141.7354)
4,-77.64846,125.245255,1.451609e+09,26.956215,409.49090,3.546830,398.87863,2016-01-01 00:35:06,POINT (-77.64846 125.245255)


In [169]:
import geopandas as gpd

In [191]:
oco_polygon_geom = shapely.geometry.Polygon([x['point'] for i,x in oco_jan1.iterrows()])
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[oco_polygon_geom])

In [192]:
import folium
m = folium.Map(list(*oco_polygon_geom.centroid.coords), zoom_start=5, tiles='cartodbpositron')
folium.GeoJson(polygon).add_to(m)
folium.LatLngPopup().add_to(m)

In [178]:
m

In [193]:
go_polygon_geom = shapely.geometry.Polygon([x['point'] for i,x in gosat_jan1.iterrows()])
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[go_polygon_geom])

In [180]:
m = folium.Map(list(*polygon_geom.centroid.coords), zoom_start=5, tiles='cartodbpositron')
folium.GeoJson(polygon).add_to(m)
folium.LatLngPopup().add_to(m)
m

In [195]:
def splitPolygon(polygon_geom):
    ls = shapely.geometry.LineString(polygon_geom.exterior.coords)
    # closed, non-simple
    lr = shapely.geometry.LineString(ls.coords[:] + ls.coords[0:1])
    mls = shapely.ops.unary_union(lr)
    mp = shapely.geometry.MultiPolygon(list(shapely.ops.polygonize(mls)))
    return mp

In [196]:
oco_polygons = splitPolygon(oco_polygon_geom)
go_polygons = splitPolygon(go_polygon_geom)

In [199]:
common_region = oco_polygons.intersection(go_polygons)